In [7]:
import torch
import pandas as pd

state_dict_file = r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Weights\resnet18_torchvision\test65_epoch598.pth"

state_dict = torch.load(state_dict_file, map_location=torch.device('cpu'))
param_list = []
for param_name, param_tensor in state_dict.items():
    temp_dict = {}
    if 'weight' in param_name: 
        # if 'conv' in param_name or 'fc' in param_name or 'classifier' in param_name:
            temp_dict["weight"] = param_name
            temp_dict["weight_shape"] = param_tensor.shape
            param_list.append(temp_dict)

df = pd.DataFrame(data=param_list, copy=False)
df_t = df.transpose()
df_t.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
weight,conv1.weight,bn1.weight,layer1.0.conv1.weight,layer1.0.bn1.weight,layer1.0.conv2.weight,layer1.0.bn2.weight,layer1.1.conv1.weight,layer1.1.bn1.weight,layer1.1.conv2.weight,layer1.1.bn2.weight,...,layer4.0.bn1.weight,layer4.0.conv2.weight,layer4.0.bn2.weight,layer4.0.downsample.0.weight,layer4.0.downsample.1.weight,layer4.1.conv1.weight,layer4.1.bn1.weight,layer4.1.conv2.weight,layer4.1.bn2.weight,fc.weight
weight_shape,"(64, 3, 7, 7)","(64,)","(64, 64, 3, 3)","(64,)","(64, 64, 3, 3)","(64,)","(64, 64, 3, 3)","(64,)","(64, 64, 3, 3)","(64,)",...,"(512,)","(512, 512, 3, 3)","(512,)","(512, 256, 1, 1)","(512,)","(512, 512, 3, 3)","(512,)","(512, 512, 3, 3)","(512,)","(10, 512)"


In [10]:
from torchvision import models
import pandas as pd

model = models.resnet34(weights=None)

model_state_dict = model.state_dict()
param_list_m = []
for param_name, param_tensor in model_state_dict.items():
    temp_dict = {}
    if 'weight' in param_name: 
        # if 'conv' in param_name or 'fc' in param_name or 'classifier' in param_name:
            temp_dict["weight"] = param_name
            temp_dict["weight_shape"] = param_tensor.shape
            param_list_m.append(temp_dict)

df1 = pd.DataFrame(data=param_list_m, copy=False)
df_t1 = df1.transpose()
df_t1.head()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
weight,conv1.weight,bn1.weight,layer1.0.conv1.weight,layer1.0.bn1.weight,layer1.0.conv2.weight,layer1.0.bn2.weight,layer1.1.conv1.weight,layer1.1.bn1.weight,layer1.1.conv2.weight,layer1.1.bn2.weight,...,layer4.0.downsample.1.weight,layer4.1.conv1.weight,layer4.1.bn1.weight,layer4.1.conv2.weight,layer4.1.bn2.weight,layer4.2.conv1.weight,layer4.2.bn1.weight,layer4.2.conv2.weight,layer4.2.bn2.weight,fc.weight
weight_shape,"(64, 3, 7, 7)","(64,)","(64, 64, 3, 3)","(64,)","(64, 64, 3, 3)","(64,)","(64, 64, 3, 3)","(64,)","(64, 64, 3, 3)","(64,)",...,"(512,)","(512, 512, 3, 3)","(512,)","(512, 512, 3, 3)","(512,)","(512, 512, 3, 3)","(512,)","(512, 512, 3, 3)","(512,)","(1000, 512)"


In [51]:
from torchvision import models
# import pandas as pd
import torch

# I named it std hahahaha.
def compare_state_dicts(dict1, dict2):
    if dict1.keys() != dict2.keys():
        # raise ValueError('Incompatible state dict and model! Unable to load weights.')
        exit('Discrepancies Detected! Unable to load weights! '
             'Make sure you have selected the correct parameter file.')
    else:
        print('Compatible State Dicts! Checking Classifier shape...')
        classifier_t = list(dict1.values())[-1]
        # print(classifier_t.shape)
        return classifier_t.shape
         



state_dict_file = r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Weights\resnet18_torchvision\test65_epoch598.pth"
state_dict = torch.load(state_dict_file, map_location=torch.device('cpu'))
model = models.resnet18(weights=None)
model_state_dict = model.state_dict()

classes = compare_state_dicts(state_dict, model_state_dict)
print(classes[0])





Compatible State Dicts! Checking Classifier shape...
10


In [57]:
import torch.nn as nn

model = models.resnet18(weights=None)

def module_fill(model):
    """Use this method to create dictionary of all available 
        CNN Conv2d and Linear layers. Commonly used to gain
        access to naming schemes of different layers in or out of
        Sequential containers. Would advise to always let this
        method fill the respective dictionary so as to reference
        model layers later on when visualizing features.

        Args:
            model: nn.Module (the model class to use)
        Returns:
            module_dict: dict (dictionary with named conv
                                and linear modules)
        """
    module_dict = {}
    for name, mod in model.named_modules():
        if len(list(mod.children())) == 0:
            if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.Linear):
                if "downsample" in name:
                    pass
                else:
                    underscored_name = name.replace(".", " ")
                    module_dict[underscored_name] = mod
    return module_dict

module_dictionary = module_fill(model)

classifier_name = list(module_dictionary.keys())[-1]

layer = getattr(model, classifier_name)
layer.out_features = 10
print(model.fc)
